In [15]:
%reload_ext autoreload
%autoreload 2

In [16]:
import os, sys

from matplotlib import pyplot as plt
from nltk.corpus import stopwords
from wordcloud import WordCloud


In [17]:
# Add parent directory to path to import modules from src
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import NewsDataLoader
import src.utils as utils